In [2]:
# Connect to workspace
from azureml.core import Workspace
ws = Workspace.from_config()

In [10]:
# Define a search space for hyperparamenter
from azureml.train.hyperdrive import GridParameterSampling, choice

param_space = {'--reg-rate': choice(0.1,0.2,0.3)}
param_sampling = GridParameterSampling(param_space) # grid search

In [7]:
# Define script configuration
from azureml.core import ScriptRunConfig, Environment

# Setup Docker environment
myenv = Environment(name="myenv")
myenv.docker.enabled = True
with open("Dockerfile", "r") as f:
    dockerfile=f.read()
myenv.docker.base_dockerfile = dockerfile
myenv.python.user_managed_dependencies = True

# Create a script config
script_config = ScriptRunConfig(source_directory='scripts',
                                script='diabetes_lr.py',
                                compute_target=ws.compute_targets['cluster'],
                                environment=myenv) 

Property base_image is mutually exclusive with base_dockerfile. Reset base_image to None


In [11]:
# Run hyperparameter opt
from azureml.core import Experiment
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal

# Configure HyperDrive
hyperdrive = HyperDriveConfig(run_config=script_config,
                              hyperparameter_sampling=param_sampling,
                              policy=None,
                              primary_metric_name='Accuracy',
                              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                              max_total_runs=6,
                              max_concurrent_runs=4)

experiment = Experiment(workspace = ws, name = 'hyperdrive_training')
hyperdrive_run = experiment.submit(config=hyperdrive)